In [1]:
import numpy as np
from numpy import log, polyfit, sqrt, std, subtract
import pandas as pd
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-darkgrid')
import seaborn as sns
from pykalman import KalmanFilter
from datetime import datetime
import statsmodels.tsa.stattools as ts
import statsmodels.api as sm
import ffn
import warnings
warnings.filterwarnings('ignore')

In [2]:
pd.read_

[*********************100%***********************]  943 of 943 completed

12 Failed downloads:
- 00279K.KS: No data found for this date range, symbol may be delisted
- 002270.KS: No data found, symbol may be delisted
- 152550.KS: No data found for this date range, symbol may be delisted
- 138250.KS: No data found, symbol may be delisted
- 016385.KS: No data found, symbol may be delisted
- 153360.KS: No data found, symbol may be delisted
- 096300.KS: No data found for this date range, symbol may be delisted
- 101060.KS: No data found, symbol may be delisted
- 007630.KS: No data found, symbol may be delisted
- 009275.KS: No data found, symbol may be delisted
- 172580.KS: No data found, symbol may be delisted
- 015350.KS: No data found, symbol may be delisted


,000020.KS,000040.KS,000050.KS,000060.KS,000070.KS,000075.KS,000080.KS,000087.KS,000100.KS,000105.KS,...,138490.KS,138930.KS,139130.KS,139480.KS,140910.KS,145990.KS,145995.KS,161000.KS,175330.KS,900140.KS
Date,,,,,,,,,,,,,,,,,,,,,
2012-01-02,3758.892578,4145.998047,6098.873047,8534.913086,78400.0,34000.0,20497.228516,13658.153320,19028.492188,13249.382812,...,4637.535645,9941.625977,10096.506836,256630.671875,4853.841309,44255.003906,29673.517578,1097.352539,3899.159180,7095.414551
2012-01-03,3800.300537,4739.997559,6125.076660,8833.075195,82500.0,35350.0,21003.814453,13658.153320,18953.871094,13249.382812,...,4619.455566,10501.717773,10177.602539,268015.406250,4861.958008,44526.781250,28929.611328,1134.893677,3891.052734,7053.429688
2012-01-04,3818.703613,5447.997070,6079.220703,8795.805664,82900.0,35300.0,21237.621094,13697.857422,19177.736328,13249.382812,...,4565.215820,10174.998047,10137.052734,265643.593750,4788.906738,44979.750000,29549.535156,1130.561768,3842.414551,7061.826660
2012-01-05,3929.124268,5213.997559,6111.975098,8646.724609,83400.0,34950.0,21783.177734,13618.449219,19550.843750,13574.123047,...,4610.415527,10361.695312,10299.247070,262323.031250,4853.841309,44662.671875,29053.599609,1113.235352,3826.201904,6969.460449
2012-01-06,3901.519043,5333.997559,6111.975098,8497.642578,83700.0,35000.0,21627.302734,13578.745117,19103.113281,13509.174805,...,4791.216309,10081.649414,10501.988281,260899.937500,4780.790039,46202.765625,28846.958984,1067.031128,3769.457275,6742.743164


In [3]:
prices

,000020.KS,000040.KS,000050.KS,000060.KS,000070.KS,000075.KS,000080.KS,000087.KS,000100.KS,000105.KS,...,138490.KS,138930.KS,139130.KS,139480.KS,140910.KS,145990.KS,145995.KS,161000.KS,175330.KS,900140.KS
Date,,,,,,,,,,,,,,,,,,,,,
2012-01-02,3758.892578,4145.998047,6098.873047,8534.913086,78400.0,34000.0,20497.228516,13658.153320,19028.492188,13249.382812,...,4637.535645,9941.625977,10096.506836,256630.671875,4853.841309,44255.003906,29673.517578,1097.352539,3899.159180,7095.414551
2012-01-03,3800.300537,4739.997559,6125.076660,8833.075195,82500.0,35350.0,21003.814453,13658.153320,18953.871094,13249.382812,...,4619.455566,10501.717773,10177.602539,268015.406250,4861.958008,44526.781250,28929.611328,1134.893677,3891.052734,7053.429688
2012-01-04,3818.703613,5447.997070,6079.220703,8795.805664,82900.0,35300.0,21237.621094,13697.857422,19177.736328,13249.382812,...,4565.215820,10174.998047,10137.052734,265643.593750,4788.906738,44979.750000,29549.535156,1130.561768,3842.414551,7061.826660
2012-01-05,3929.124268,5213.997559,6111.975098,8646.724609,83400.0,34950.0,21783.177734,13618.449219,19550.843750,13574.123047,...,4610.415527,10361.695312,10299.247070,262323.031250,4853.841309,44662.671875,29053.599609,1113.235352,3826.201904,6969.460449
2012-01-06,3901.519043,5333.997559,6111.975098,8497.642578,83700.0,35000.0,21627.302734,13578.745117,19103.113281,13509.174805,...,4791.216309,10081.649414,10501.988281,260899.937500,4780.790039,46202.765625,28846.958984,1067.031128,3769.457275,6742.743164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-14,9270.000000,593.000000,11050.000000,34250.000000,68600.0,56500.0,27100.000000,16500.000000,58000.000000,56500.000000,...,10700.000000,6760.000000,6940.000000,93000.000000,4090.000000,38900.000000,31050.000000,10500.000000,7430.000000,2975.000000
2022-11-15,9550.000000,588.000000,10950.000000,34700.000000,69200.0,56200.0,26850.000000,16650.000000,59400.000000,56800.000000,...,10750.000000,6820.000000,7050.000000,93600.000000,4135.000000,39100.000000,31150.000000,9880.000000,7500.000000,3030.000000
2022-11-16,9390.000000,583.000000,11000.000000,34650.000000,68600.0,57000.0,26950.000000,16600.000000,59100.000000,56900.000000,...,10900.000000,6900.000000,7290.000000,95700.000000,4150.000000,39050.000000,31600.000000,9610.000000,7720.000000,3020.000000


In [4]:
def find_cointegrated_pairs(dataframe, critial_level = 0.05):
    # Length of dateframe
    n = dataframe.shape[1] 
    # Initialise the matrix of p
    pvalue_matrix = np.ones((n, n)) 
    # Get the column names
    keys = dataframe.keys() 
    pairs = []
    for i in range(n):
        # For j bigger than i
        for j in range(i+1, n): 
            # Obtain the price of two contracts
            stock1 = dataframe[keys[i]] 
            stock2 = dataframe[keys[j]]
            # Get cointegration
            result = sm.tsa.stattools.coint(stock1, stock2) 
            # Get the pvalue
            pvalue = result[1] 
            pvalue_matrix[i, j] = pvalue
            # If p-value is less than the critical level
            if pvalue < critial_level: 
                row = [keys[i], keys[j], pvalue]
                # Record the contract with that p-value
                pairs.append(row) 
    pairs = pd.DataFrame(pairs,columns = ['S1', 'S2', 'Pvalue'])
    return (pvalue_matrix, pairs)

pvalue_matrix, pairs = find_cointegrated_pairs(prices)

print(pairs)

MissingDataError: exog contains inf or nans